In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
# %matplotlib notebook
%matplotlib qt
def recoverTransformation(Image1, Image2, n):
    assert len(Image1.shape) == len(Image2.shape), 'Image1 and Image2 must have same channels'  
    # Draw atleast 3 points on both images in same order, i.e kth point marked should be on the same position in both images
    plt.imshow(Image1)
    image1_points = plt.ginput(n)
    plt.show()
    plt.close()

    plt.imshow(Image2)
    image2_points = plt.ginput(n)
    plt.close()
    X = np.array([[[pt[0], pt[1], 1, 0, 0, 0], [0, 0, 0, pt[0], pt[1], 1]] for pt in image2_points])
    c = X.shape[0]
    X = X.reshape(2*c,6)
    
    x_prime = []
    for pt in image1_points:
        x_prime.append(pt[0])
        x_prime.append(pt[1])
    x_prime = np.array(x_prime)

    X_pseudo = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.transpose(X))
    T = np.matmul(X_pseudo, x_prime)
    T = T.reshape(2,3)
    rows, cols, = Image1.shape[0], Image1.shape[1]
    T_Image = cv2.warpAffine(Image2,T,(cols,rows))
    
    A, t = T[:2, :2], [T[0,2], T[1,2]]
    T_points = []
    for p in image2_points:
        t_p = np.dot(A,p)+t
        T_points.append(t_p)
    MSEPix = (np.square(np.array(image1_points) - np.array(T_points))).mean()
    MSECorPts = (np.square(Image1 - T_Image)).mean()
    return [MSEPix, MSECorPts, T, T_Image]

In [2]:
def registerImages(ImageNew, ImageOld):

   
    # recovering transformation
    MSEPix, MSECorPts, T, T_Image= recoverTransformation(ImageNew, ImageOld, 5)
    print ("The mean squared error of pixels of T_image and Image2" , MSEPix )
    print("The mean squared error of correspondence points and transformed points", MSECorPts)
    print(" Recovered transformation" , T)
    # Masking
    indices = T_Image!=0
    registeredImage = np.copy(ImageNew)
    registeredImage[indices] = T_Image[indices]
    return registeredImage

In [3]:
ImageNew = plt.imread('dataset/badshahiNew.jpg')
ImageOld = plt.imread('dataset/badshahiOld.jpg')
#ImageNew = cv2.cvtColor(ImageNew, cv2.COLOR_BGR2GRAY)
registeredImage = registerImages(ImageNew, ImageOld)
cv2.imwrite('Results/Badshahi_registered_image.png', registeredImage)
plt.imshow(registeredImage)

The mean squared error of pixels of T_image and Image2 185.35753791218264
The mean squared error of correspondence points and transformed points 99.04440048993644
 Recovered transformation [[ 1.72373683e+00 -9.43749060e-03  1.96083686e+02]
 [ 3.22864330e-02  1.06132528e+00  6.51826874e+01]]
